In [3]:
!pip install groq qdrant-client
!pip install langchain-groq
!pip install langchainhub
!pip install PyPDF2 # install the missing package
!pip install langchain-community
!pip install gradio
!pip install fuzzywuzzy
!pip install transformers
!pip install sentence_transformers
!pip install langchain-text-splitter
!pip install openai
!pip install python-dotenv
!pip install llama-cpp-python
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━

# Imports

In [4]:
import os
from PyPDF2 import PdfReader
import numpy as np
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
import numpy as np
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import Tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.llms import OpenAI
from langchain_groq import ChatGroq
import gradio as gr
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Extracting Text from PDFs

In [5]:
def extract_text_from_pdf(pdf_path):
    """
    Extract text content from a PDF file.

    Args:
    - pdf_path (str): The path to the PDF file.

    Returns:
    - str: The extracted text content.
    """
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for page in reader.pages:
        extracted_text += page.extract_text()
    return extracted_text

def extract_text_from_pdfs_in_directory(directory):
    """
    Extract text content from all PDF files in a directory and save as text files.

    Args:
    - directory (str): The path to the directory containing PDF files.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            extracted_text = extract_text_from_pdf(pdf_path)
            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_filepath = os.path.join(directory, txt_filename)
            with open(txt_filepath, "w") as txt_file:
                txt_file.write(extracted_text)

# Specify the directory containing PDF files
directory_path = "Docs/"

# Extract text from PDFs in the directory and save as text files
extract_text_from_pdfs_in_directory(directory_path)


# Generating Embedding

In [19]:
GROQ_API_KEY = "<YOUR_GROQ_API_KEY >"

In [7]:
# Step 1: List all .txt files in the directory
directory_path = "/content/Docs"
txt_files = [file for file in os.listdir(directory_path) if file.endswith('.txt')]
print(txt_files)

['Facebook Terms of Usage.txt', 'OPENAI Terms of use.txt', 'SLSHandbook Contract Law.txt', 'Google Terms of Service_en_in.txt']


In [8]:
all_documents = {}
for txt_file in txt_files:
    loader = TextLoader(os.path.join(directory_path, txt_file))
    documents = loader.load()

    # Step 2: Split documents into chunks and add metadata
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator="\n")
    docs = text_splitter.split_documents(documents)
    for doc in docs:
        doc.metadata["source"] = txt_file  # Add source metadata

    all_documents[txt_file] = docs

In [9]:
# Initialize the TextEmbedding model
from langchain.embeddings import HuggingFaceEmbeddings

# Step 3: Initialize the TextEmbedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Step 4: Create Qdrant vector store collections for each document
qdrant_collections = {}
for txt_file in txt_files:
    qdrant_collections[txt_file] = Qdrant.from_documents(
        all_documents[txt_file],
        embeddings,
        location=":memory:",
        collection_name=txt_file,
    )

In [11]:
for txt_file in txt_files:
    print(f"Collection: {qdrant_collections[txt_file].collection_name}")

Collection: Facebook Terms of Usage.txt
Collection: OPENAI Terms of use.txt
Collection: SLSHandbook Contract Law.txt
Collection: Google Terms of Service_en_in.txt


In [12]:
retriever = {}
for txt_file in txt_files:
    retriever[txt_file] = qdrant_collections[txt_file].as_retriever()

# Setting ReAct Agents

In [13]:
def get_age(name: str, person_database: dict) -> int:
    """
    Get the age of a person from the database.

    Args:
    - name (str): The name of the person.
    - person_database (dict): A dictionary containing person information.

    Returns:
    - int: The age of the person if found, otherwise None.
    """
    if name in person_database:
        return person_database[name]["Age"]
    else:
        return None


def get_age_info(name: str) -> str:
    """
    Get age and health information for a person.

    Args:
    - name (str): The name of the person.

    Returns:
    - str: A string containing age and health information for the person.
    """
    person_database = {
        "Sam": {"Age": 21, "Nationality": "US"},
        "Alice": {"Age": 25, "Nationality": "UK"},
        "Bob": {"Age": 11, "Nationality": "US"}
    }
    age = get_age(name, person_database)
    if age is not None:
        return f"\nAge: {age}\n"
    else:
        return f"\nAge Information for {name} not found.\n"



def get_today_date(input : str) -> str:
    import datetime
    today = datetime.date.today()
    return f"\n {today} \n"



def get_relevant_document(name : str) -> str:
    # String name for fuzzy search
    search_name = name

    # Find the best match using fuzzy search
    best_match = process.extractOne(search_name, txt_files, scorer=fuzz.ratio)

    # Get the selected file name
    selected_file = best_match[0]

    selected_retriever = retriever[selected_file]

    global query
    results = selected_retriever.get_relevant_documents(query)
    global retrieved_text

    total_content = "\n\nBelow are the related document's content: \n\n"
    chunk_count = 0
    for result in results:
        chunk_count += 1
        if chunk_count > 4:
            break
        total_content += result.page_content + "\n"
    retrieved_text = total_content
    return total_content


def get_summarized_text(name : str) -> str:
    from transformers import pipeline
    summarizer = pipeline("summarization", model="Falconsai/text_summarization")
    global retrieved_text
    article = retrieved_text
    return summarizer(article, max_length=1000, min_length=30, do_sample=False)[0]['summary_text']


# Define the Tool
get_age_info_tool = Tool(
    name="Get Age",
    func=get_age_info,
    description="Useful for getting age information for any person. Input should be the name of the person."
)

get_today_date_tool = Tool(
    name="Get Todays Date",
    func=get_today_date,
    description="Useful for getting today's date"
)

get_relevant_document_tool = Tool(
    name="Get Relevant document",
    func=get_relevant_document,
    description="Useful for getting relevant document that we need."
)

get_summarized_text_tool = Tool(
    name="Get Summarized Text",
    func=get_summarized_text,
    description="Useful for getting summarized text for any document."
)

# Common Agent prompts

In [14]:
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


# Running the ReAct agent

In [22]:
retrieved_text = ""
tools = [get_relevant_document_tool, get_summarized_text_tool, get_today_date_tool, get_age_info_tool]

model = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY, temperature=0)
# model = OpenAI(openai_api_key="YOUR_OPEN_AI_KEY")

react_agent = create_react_agent(model, tools=tools, prompt=prompt_react)
react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [23]:
query = "Give me the summary for the question : What age requirement is specified for using the OpenAI Services, and what provision applies if the user is under 18?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: I need to find a relevant document that mentions the age requirement for using OpenAI Services.

Action: Get Relevant document
Action Input: OpenAI Terms of Service

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




Below are the related document's content: 

Who we are
OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. For more information about OpenAI, please visit https://openai.com/about.
Registration and access
Minimum age. You must be at least 13 years old or the minimum age required in your country to consent to use the Services. If you are under 18 you must have your parent or legal guardian’s permission to use the Services. 
Registration. You must provide accurate and complete information to register for an account to use our Services. You may not share your account credentials or make your account available to anyone else and are responsible for all activities that occur under your account. If you create an account or use the Services on behalf of another person or entity, you must have the authority to accept these Terms on their behalf.
Using our Services
Terms of use
Effective: January 31, 2024 (pr

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (732 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 732. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=366)


Who we are OpenAI is an AI research and deployment company . You must be at least 13 years old or the minimum age required in your country to consent to use the Services . If you are under 18, you must have your parent or legal guardian’s permission to use our Services. You may not share your account credentials or make your account available to anyone else . This Terms of Use apply to your use of ChatGPT Enterprise, our APIs, and our other services for businesses and developers .Thought: I have the summarized text, now I need to extract the specific information about the age requirement.
Invalid Format: Missing 'Action:' after 'Thought:I apologize for the mistake. Here is the corrected response:

Thought: I have the summarized text, now I need to extract the specific information about the age requirement.

Action: Get Age
Action Input: OpenAI Services age requirement
Age Information for OpenAI Services age requirement not found.
Thought: I realize that I don't need to use the Get Age 

{'input': 'Give me the summary for the question : What age requirement is specified for using the OpenAI Services, and what provision applies if the user is under 18?',
 'output': "The age requirement for using OpenAI Services is at least 13 years old or the minimum age required in your country to consent to use the Services. If you are under 18, you must have your parent or legal guardian's permission to use the Services."}

In [24]:
query = "Give me summary of What resources does Google offer to users for assistance and guidance in using its services?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: I need to get a relevant document that talks about Google's resources for user assistance and guidance.

Action: Get Relevant document
Action Input: "Google user resources"

Below are the related document's content: 

services — whether that content belongs to you, Google, or others
In case of problems or disagreements, which describes other legal rights you have, and what to expect
in case someone violates these terms
Understanding these terms is important because, by using our services, you’re agreeing to these terms.
Besides these terms, we also publish a Privacy Policy. Although it’s not part of these terms, we encourage
you to read it to better understand how you can update, manage, export, and delete your information.
Terms
Service providerGoogle services are provided by, and you’re contracting with:
Google LLC
organized under the laws of the State of Delaware, USA, and operating under the laws of the USA
1600 Amphitheatre Parkway


Token indices sequence length is longer than the specified maximum sequence length for this model (805 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 805. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=402)


This Privacy Policy is important because, by using our services, you’re agreeing to these terms . You’re responsible for what you do with your Google Account . To use our services on behalf of an organization, we sometimes send you service announcements and other information . These Terms of Service help dene Google’s relationship with you as you interact with our services .Thought: I now know the summarized text, but I need to extract the resources that Google offers to users for assistance and guidance in using its services.

Action: Get Summarized Text
Action Input: The summarized text

Token indices sequence length is longer than the specified maximum sequence length for this model (805 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 805. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=402)


This Privacy Policy is important because, by using our services, you’re agreeing to these terms . You’re responsible for what you do with your Google Account . To use our services on behalf of an organization, we sometimes send you service announcements and other information . These Terms of Service help dene Google’s relationship with you as you interact with our services .Thought: I need to extract the resources that Google offers to users for assistance and guidance in using its services from the summarized text.

Action: Get Summarized Text
Action Input: The summarized text

Token indices sequence length is longer than the specified maximum sequence length for this model (805 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 805. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=402)


This Privacy Policy is important because, by using our services, you’re agreeing to these terms . You’re responsible for what you do with your Google Account . To use our services on behalf of an organization, we sometimes send you service announcements and other information . These Terms of Service help dene Google’s relationship with you as you interact with our services .Thought: I now know the final answer.

Final Answer: Google offers the following resources to users for assistance and guidance in using its services: Terms of Service, Privacy Policy, Security Checkup, and service announcements and other information. These resources help users understand how to use Google services, manage their accounts, and stay informed about Google's services and policies.

> Finished chain.


{'input': 'Give me summary of What resources does Google offer to users for assistance and guidance in using its services?',
 'output': "Google offers the following resources to users for assistance and guidance in using its services: Terms of Service, Privacy Policy, Security Checkup, and service announcements and other information. These resources help users understand how to use Google services, manage their accounts, and stay informed about Google's services and policies."}

In [25]:
query = "What are my rights to my data on Facebook?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: I need to find a relevant document that outlines Facebook's data policy.

Action: Get Relevant document
Action Input: Facebook Data Policy

Below are the related document's content: 

viii. Processing friend lists from Facebook to establish social connections in your App unless each person in that connection has granted you access to that information for that purpose.
b. Additional Terms for Restricted Platform Data
i. You will not request Restricted Platform Data unless it is necessary to meaningfully improve the quality of the applicable User's experience in the specific product or service for which the User shared the data.
ii. It must be clear to the User why you are requesting their Restricted Platform Data in order to improve the quality of their experience.
iii. For clarity, your Processing of Restricted Platform Data must comply with the applicable Developer Docs and other provisions of these Terms (including the Prohibited Pract

Token indices sequence length is longer than the specified maximum sequence length for this model (803 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 803. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=401)


You will not request Restricted Platform Data unless it is necessary to meaningfully improve the quality of the applicable User's experience in the specific product or service for which the User shared the data . You must comply with the applicable Developer Docs and other provisions of these Terms (including the Prohibited Practices) You may only share Platform Data in compliance with these Terms . We may require that your Clients agree to these Terms or other applicable terms or policies .Thought: I have a summarized version of the Facebook Data Policy document, now I need to understand what it means for my rights to my data on Facebook.

Action: None
Action Input: NoneNone is not a valid tool, try one of [Get Relevant document, Get Summarized Text, Get Todays Date, Get Age].Thought: I understand that the summarized text is outlining the rules and regulations that developers and clients must follow when it comes to processing and sharing platform data on Facebook. However, I still ne

Token indices sequence length is longer than the specified maximum sequence length for this model (803 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 803. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=401)


You will not request Restricted Platform Data unless it is necessary to meaningfully improve the quality of the applicable User's experience in the specific product or service for which the User shared the data . You must comply with the applicable Developer Docs and other provisions of these Terms (including the Prohibited Practices) You may only share Platform Data in compliance with these Terms . We may require that your Clients agree to these Terms or other applicable terms or policies .Thought: I need to rephrase the observation to focus on user rights.

Action: Get Summarized Text
Action Input: The provided Facebook Data Policy document, focusing on user rights

Token indices sequence length is longer than the specified maximum sequence length for this model (803 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 803. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=401)


You will not request Restricted Platform Data unless it is necessary to meaningfully improve the quality of the applicable User's experience in the specific product or service for which the User shared the data . You must comply with the applicable Developer Docs and other provisions of these Terms (including the Prohibited Practices) You may only share Platform Data in compliance with these Terms . We may require that your Clients agree to these Terms or other applicable terms or policies .Thought: I need to rephrase the observation to focus on user rights in a more concise manner.

Action: Get Summarized Text
Action Input: The provided Facebook Data Policy document, focusing on user rights

Token indices sequence length is longer than the specified maximum sequence length for this model (803 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but your input_length is only 803. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=401)


You will not request Restricted Platform Data unless it is necessary to meaningfully improve the quality of the applicable User's experience in the specific product or service for which the User shared the data . You must comply with the applicable Developer Docs and other provisions of these Terms (including the Prohibited Practices) You may only share Platform Data in compliance with these Terms . We may require that your Clients agree to these Terms or other applicable terms or policies .Thought: I think I have finally understood the user rights from the Facebook Data Policy document.

Action: None
Action Input: NoneNone is not a valid tool, try one of [Get Relevant document, Get Summarized Text, Get Todays Date, Get Age].Thought: I think I have finally understood the user rights from the Facebook Data Policy document.

Final Answer: As a user, my rights to my data on Facebook include the right to have my data processed only when necessary to improve my experience in a specific prod

{'input': 'What are my rights to my data on Facebook?',
 'output': "As a user, my rights to my data on Facebook include the right to have my data processed only when necessary to improve my experience in a specific product or service, and the right to have my data shared only in compliance with Facebook's Terms and applicable laws and regulations."}

In [26]:
query = "I am Bob. Will i be eligible in 2027 for the age requirement specified for using the OpenAI Services by OpenAI Terms?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: I need to know the age requirement specified in the OpenAI Terms to determine if Bob will be eligible in 2027.

Action: Get Relevant document
Action Input: OpenAI Terms

Below are the related document's content: 

Who we are
OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. For more information about OpenAI, please visit https://openai.com/about.
Registration and access
Minimum age. You must be at least 13 years old or the minimum age required in your country to consent to use the Services. If you are under 18 you must have your parent or legal guardian’s permission to use the Services. 
Registration. You must provide accurate and complete information to register for an account to use our Services. You may not share your account credentials or make your account available to anyone else and are responsible for all activities that occur under your accoun

{'input': 'I am Bob. Will i be eligible in 2027 for the age requirement specified for using the OpenAI Services by OpenAI Terms?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [27]:
import gradio as gr
from io import StringIO
import sys
import re

def generate_response(question):
    """
    Generate a response based on the provided question using ChatGroq.

    Args:
    - question (str): The question input by the user.

    Returns:
    - str: The generated response based on the question.
    """
    tools = [get_relevant_document_tool, get_summarized_text_tool, get_today_date_tool, get_age_info_tool]

    model = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY, temperature=0)
    # model = OpenAI(openai_api_key="<YOUR_OPENAI>")

    react_agent = create_react_agent(model, tools=tools, prompt=prompt_react)
    react_agent_executor = AgentExecutor(
        agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
    )

    # Redirect stdout to capture text
    with StringIO() as text_output:
        sys.stdout = text_output
        completion = react_agent_executor.invoke({"input": question})
        sys.stdout = sys.__stdout__  # Reset stdout

        # Get the captured text
        text_output_str = text_output.getvalue()

    # Remove ANSI escape codes
    text_output_str = re.sub(r'\x1b\[[0-9;]*m', '', text_output_str)

    return text_output_str

# Set up the Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=[gr.Textbox(label="Question")],  # Pass input as a list
    outputs=[gr.Textbox(label="Generated Response")],  # Pass output as a list
    title="Intellegent RAG with Qdrant, LangChain ReAct and Llama3 from Groq Endpoint",
    description="Enter a question and get a generated response based on the retrieved text.",
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://43c079a31bd3c54418.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
